# Attempt 1
This notebook is meant to read the Citibike dataset and test out simple ML algorithms with a subset of the original data.

## Reading the Dataset
The data being used is only for *Jan 2024*. <br><br>
<u>An important note</u>: the dataset contains both *electric_bike* and *classic_bike*. The way Citibike deploys these two vehicle types **differs,** as explained in this notebook.

In [1]:
import pandas as pd
citi_jan_24 = pd.read_csv("C:/Users/singh/Desktop/TUD (All Semesters)/Courses - Semester 5 (TU Dresden)/Research Task - Spatial Modelling/Datasets/Citibike Trip Data/202401-citibike-tripdata.csv")
citi_jan_24.head()

C:\Users\singh\AppData\Local\Temp\ipykernel_11940\4181878887.py:2: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  citi_jan_24 = pd.read_csv("C:/Users/singh/Desktop/TUD (All Semesters)/Courses - Semester 5 (TU Dresden)/Research Task - Spatial Modelling/Datasets/Citibike Trip Data/202401-citibike-tripdata.csv")


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,5078F3D302000BD2,electric_bike,2024-01-22 18:43:19.012,2024-01-22 18:48:10.708,Frederick Douglass Blvd & W 145 St,7954.12,St Nicholas Ave & W 126 St,7756.10,40.823072,-73.941738,40.811432,-73.951878,member
1,814337105D37302A,electric_bike,2024-01-11 19:19:18.721,2024-01-11 19:47:36.007,W 54 St & 6 Ave,6771.13,E 74 St & 1 Ave,6953.08,40.761822,-73.977036,40.768974,-73.954823,member
2,A33A920E2B10710C,electric_bike,2024-01-30 19:17:41.693,2024-01-30 19:32:49.857,E 11 St & Ave B,5659.11,W 10 St & Washington St,5847.06,40.727592,-73.979751,40.733424,-74.008515,casual
3,A3A5FC0DD7D34D74,electric_bike,2024-01-27 11:27:01.759,2024-01-27 11:38:01.213,W 54 St & 6 Ave,6771.13,E 74 St & 1 Ave,6953.08,40.761779,-73.977144,40.768974,-73.954823,member
4,6F96728ECEFBDAA4,electric_bike,2024-01-16 15:15:41.000,2024-01-16 15:29:26.156,Madison Ave & E 99 St,7443.01,E 74 St & 1 Ave,6953.08,40.789808,-73.952214,40.768974,-73.954823,member


In [2]:
# Total rides in the dataset
len(citi_jan_24)

1888085

In [3]:
# Number of unique stations from which the rides began
len(citi_jan_24["start_station_name"].unique())

2129

In [4]:
# Number of unique stations where the rides ended
len(citi_jan_24["end_station_name"].unique())

2167

This suggests that rides are beggining from limited areas but are ending in diverse areas.

In [5]:
# Cross-checking the same result with station ids
print(len(citi_jan_24["start_station_id"].unique()),
len(citi_jan_24["end_station_id"].unique()))

4254 2346


**There is a problem: how can there be more ids and less stations?**

In [6]:
# Troubleshooting
citi_jan_24[citi_jan_24["start_station_name"] == "Frederick Douglass Blvd & W 145 St"]["start_station_id"].unique()

array(['7954.12', 7954.12], dtype=object)

**The problem is some ids are stored as *str* while others are stored as *int*.**<br>
So, it's better to use <u>station names</u>. The station numbers are also mentioned in the *monthly report*.

[Citibike Monthly Report - January 2024](https://mot-marketing-whitelabel-prod.s3.amazonaws.com/nyc/January-2024-Citi-Bike-Monthly-Report.pdf)

In [10]:
# There are a lot of missing values in the complete dataset
citi_jan_24["start_station_name"].sort_values()

1542986    1 Ave & E 110 St
334007     1 Ave & E 110 St
269164     1 Ave & E 110 St
269167     1 Ave & E 110 St
203983     1 Ave & E 110 St
                 ...       
1831375                 NaN
1848253                 NaN
1848254                 NaN
1848297                 NaN
1848315                 NaN
Name: start_station_name, Length: 1888085, dtype: object

In [11]:
# Number of missing values
sum(citi_jan_24["start_station_name"].isna())

1160

In [27]:
# Viewing rows with 'nan' start_station_name
citi_jan_24[citi_jan_24["start_station_name"].isna()]

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
3505,968353875D9E171C,electric_bike,2024-01-15 14:40:10.123,2024-01-15 14:54:09.164,NaN,NaN,Central Park North & Adam Clayton Powell Blvd,7617.07,40.78,-73.95,40.799484,-73.955613,member
3520,4D211FBABE06A67E,electric_bike,2024-01-04 00:59:31.969,2024-01-04 01:24:51.962,NaN,NaN,Henry St & Atlantic Ave,4531.05,40.76,-73.98,40.690893,-73.996123,member
3523,CE8C18D21E7C4906,electric_bike,2024-01-21 17:50:16.612,2024-01-21 18:00:08.384,NaN,NaN,Rockaway Ave & Bainbridge St,4285.09,40.70,-73.91,40.683200,-73.911666,member
3525,8E38EB66FE53F7DC,electric_bike,2024-01-19 08:50:16.377,2024-01-19 09:11:31.579,NaN,NaN,Henry St & Atlantic Ave,4531.05,40.73,-73.99,40.690893,-73.996123,member
3567,F6FA5C6723898936,electric_bike,2024-01-15 15:51:20.528,2024-01-15 15:55:08.884,NaN,NaN,Henry St & Atlantic Ave,4531.05,40.70,-73.99,40.690893,-73.996123,casual
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1831375,912D0635471F1A63,electric_bike,2024-01-23 10:32:47.362,2024-01-23 10:45:19.915,NaN,NaN,Bulova Ave & Brooklyn Queens Expressway W,6766.03,40.74,-73.91,40.762507,-73.902534,member
1848253,984F5B8FBA65C5D3,electric_bike,2024-01-10 16:48:54.484,2024-01-10 16:58:34.171,NaN,NaN,Fort Washington Ave & W 173 St,8320.01,40.83,-73.94,40.845220,-73.940493,member
1848254,58717196C5E8D8C4,electric_bike,2024-01-22 17:36:46.735,2024-01-22 17:38:41.520,NaN,NaN,14 St & 7 Ave,3731.11,40.67,-73.98,40.663779,-73.983968,member
1848297,0134322C09F09464,electric_bike,2024-01-02 19:14:36.857,2024-01-02 19:20:16.723,NaN,NaN,E 65 St & 2 Ave,6860.12,40.76,-73.97,40.764719,-73.962221,casual


In [28]:
# Stations are not defined only where 'electric bikes' are used
citi_jan_24[citi_jan_24["start_station_name"].isna()]["rideable_type"].unique()

array(['electric_bike'], dtype=object)

In [30]:
# If the dataset is filtered for 'classic bikes' all starting stations should be visible
sum(citi_jan_24[citi_jan_24["rideable_type"] == "classic_bike"]["start_station_name"].isna())

0

Hence there are no *NaN* values for **classic_bikes** in our dataset, for the column *start_station_name*.

In [33]:
# If the dataset is filtered for 'classic bikes' all ending stations should also be visible
sum(citi_jan_24[citi_jan_24["rideable_type"] == "classic_bike"]["end_station_name"].isna())

498

Even with **classic_bike** we still have missing data for end_station_name.

In [41]:
# Filtering the dataset only for classic_bike
cbike_j24 = citi_jan_24[citi_jan_24["rideable_type"] == "classic_bike"]
cbike_j24.info()

<class 'pandas.core.frame.DataFrame'>
Index: 673559 entries, 15 to 1888083
Data columns (total 13 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   ride_id             673559 non-null  object 
 1   rideable_type       673559 non-null  object 
 2   started_at          673559 non-null  object 
 3   ended_at            673559 non-null  object 
 4   start_station_name  673559 non-null  object 
 5   start_station_id    673559 non-null  object 
 6   end_station_name    673061 non-null  object 
 7   end_station_id      673061 non-null  object 
 8   start_lat           673559 non-null  float64
 9   start_lng           673559 non-null  float64
 10  end_lat             673080 non-null  float64
 11  end_lng             673080 non-null  float64
 12  member_casual       673559 non-null  object 
dtypes: float64(4), object(9)
memory usage: 71.9+ MB


In [38]:
# Viewing rows with 'nan' end_station_name, only for 'classic_bike'
cbike_j24[cbike_j24["end_station_name"].isna()]

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
38,03DF018B20ED02D9,classic_bike,2024-01-13 14:51:04.773,2024-01-14 15:50:59.897,Columbia St & Kane St,4422.05,NaN,NaN,40.687632,-74.001626,NaN,NaN,casual
66,7B1D53129AD9425A,classic_bike,2024-01-13 13:49:53.172,2024-01-14 14:49:40.126,E 16 St & Irving Pl,5938.11,NaN,NaN,40.735367,-73.987974,NaN,NaN,member
90,6EF6EA88DF453151,classic_bike,2024-01-25 02:58:50.887,2024-01-26 03:58:37.719,Schermerhorn St & Hoyt St,4479.10,NaN,NaN,40.688626,-73.985191,NaN,NaN,member
123,478448F64501E873,classic_bike,2024-01-13 17:47:17.212,2024-01-14 18:47:12.403,8 Ave & W 27 St,6297.07,NaN,NaN,40.747968,-73.996637,NaN,NaN,member
136,49D93C7E1ED3CEB6,classic_bike,2024-01-08 14:59:44.611,2024-01-09 15:59:36.472,Parkside Ave & Ocean Ave,3448.02,NaN,NaN,40.654850,-73.962170,NaN,NaN,casual
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1884619,B8451B6611952144,classic_bike,2024-01-30 16:11:20.102,2024-01-31 17:10:59.528,E 76 St & Park Ave,7128.08,NaN,NaN,40.773763,-73.962221,NaN,NaN,casual
1884678,54BCFB6628427129,classic_bike,2024-01-04 01:00:23.239,2024-01-05 02:00:14.302,Carmine St & 6 Ave,5763.03,NaN,NaN,40.730386,-74.002150,NaN,NaN,casual
1884699,3347A06314987874,classic_bike,2024-01-23 11:20:25.283,2024-01-24 12:20:21.046,E 76 St & Park Ave,7128.08,NaN,NaN,40.773763,-73.962221,NaN,NaN,member
1884833,242577697DC24F3B,classic_bike,2024-01-27 11:03:08.809,2024-01-28 12:02:52.559,E 76 St & Park Ave,7128.08,NaN,NaN,40.773763,-73.962221,40.78,-73.95,casual


In [31]:
# Number of unique starting stations for classic_bikes
len(pd.Series(cbike_j24["start_station_name"].unique()))

2119

In [32]:
# Number of unique ending stations for classic_bikes
len(pd.Series(cbike_j24["end_station_name"].unique()))

2137

The approach to subset the dataset based on **rideable_type** may/ may not work. Look further into this when the dataset is better understood and you know how to better evaluate *demand*.

## Time Resolution
It would be useful to see at what level of time accuracy is the dataset capturing the ridership information. Here, complete **unfiltered dataset** is being studied.

In [42]:
# No fixed time resolution
citi_jan_24["started_at"].sort_values()

243902     2023-12-31 02:36:55.648
243901     2023-12-31 06:25:55.501
20501      2023-12-31 08:43:59.561
245456     2023-12-31 08:59:42.045
97370      2023-12-31 09:50:12.745
                    ...           
152866     2024-01-31 23:55:59.368
1183902    2024-01-31 23:56:38.776
291989     2024-01-31 23:57:04.310
1041975    2024-01-31 23:57:04.650
315390     2024-01-31 23:58:30.270
Name: started_at, Length: 1888085, dtype: object

The time accuracy depends on when the ride took place. In busy hours, the accuracy is up to a second.

In [43]:
# Extracting time based features
citi_jan_24["started_at"][0]

'2024-01-22 18:43:19.012'

In [46]:
# Converting to datetime object
timestamp_test = pd.to_datetime(citi_jan_24["started_at"][0])
timestamp_test

Timestamp('2024-01-22 18:43:19.012000')

In [50]:
# extracting date elements
print(timestamp_test.year, timestamp_test.day, timestamp_test.month)

2024 22 1


In [57]:
# extracting more info from date features
print(timestamp_test.day_name(), timestamp_test.month_name())

Monday January


In [63]:
# extracting all time elements
print(timestamp_test.hour, timestamp_test.minute, timestamp_test.second, timestamp_test.microsecond)

18 43 19 12000


Using these time methods, you can create date-time features for the existing dataset. *Date Range* can also be created that can help with determining demand, for example a **15 minute** window.

In [67]:
# there might be pre-loaded citibike dataset as well
import mglearn
citibike = mglearn.datasets.load_citibike()

In [69]:
citibike

starttime
2015-08-01 00:00:00     3
2015-08-01 03:00:00     0
2015-08-01 06:00:00     9
2015-08-01 09:00:00    41
2015-08-01 12:00:00    39
                       ..
2015-08-31 09:00:00    16
2015-08-31 12:00:00     8
2015-08-31 15:00:00    17
2015-08-31 18:00:00    22
2015-08-31 21:00:00     7
Freq: 3h, Name: one, Length: 248, dtype: int64